In [4]:
from datetime import datetime,date
import logging
import threading
from queue import Queue
from kiteconnect import KiteTicker, KiteConnect
import time
from collections import defaultdict, deque
import csv
import os
import pandas as pd
from Bardata import BarData
import dotenv
from dotenv import load_dotenv,set_key
import argparse
from black_scholes_model import option_greeks
load_dotenv()

True

In [ ]:
class DeltaStrategy():

    def __init__(self, config):
        self.config = config
        self.api_key = os.getenv('api_key')
        self.access_token = os.getenv('access_token')
        self.kws = KiteTicker(self.api_key, self.access_token)
        self.kite = KiteConnect(api_key=self.api_key)
        self.instruments = pd.DataFrame(self.kite.instruments(self.config['exchange']))

        self.index_tokens = {'NIFTY':256265, 'SENSEX': 265}
        self.instrument_tokens = [self.index_tokens[self.config['trading_symbol']]]

        self.latest_ticks = {self.index_tokens[self.config['trading_symbol']]: None}
        self.strikes = {}
        self.trading_symbols = {}
        self.expiry = None


        self.tick_queue = Queue()
        self.lock = threading.Lock()

        self.iv = {}
        self.delta = {}
        self.gamma = {}


        return

    def process_ticks(self):
        while True:
            try:
                tick = self.tick_queue.get(timeout=1)
                with self.lock:
                    # Process the tick data
                    print(f"Processing tick: {tick['last_price']}")
                    if tick['instrument_token'] in self.index_tokens.values():
                        self.latest_ticks[tick['instrument_token']] = tick
                        continue
                    
                    tick['trading_symbol'] = self.trading_symbols[tick['instrument_token']]
                    time_to_expiry = (self.expiry - tick['exchange_timestamp']).total_seconds() / (365*86400)  # Convert to days

                    iv, delta, gamma = option_greeks(
                        spot=self.latest_ticks[self.index_tokens[self.config['trading_symbol']]]['last_price'],
                        strike=self.strikes[tick['instrument_token']],
                        dte=time_to_expiry,
                        rate=0.06,  # Example risk-free rate
                        option_type='CE' if 'CE' in tick['tradingsymbol'] else 'PE',
                        option_price=tick['last_price']
                    )

                    self.iv[self.trading_symbols[tick['instrument_token']]] = iv
                    self.delta[tick['trading_symbol']] = delta
                    self.gamma[tick['trading_symbol']] = gamma

                    tick['iv'] = iv
                    tick['delta'] = delta
                    tick['gamma'] = gamma

                    self.latest_ticks[tick['instrument_token']] = tick

                    
                    pass

            except Exception as e:
                print(f"Error processing tick: {e}")    
    
    def update_trading_symbol(self):
        expiry = sorted(self.instruments.loc[self.instruments['name']==self.config['trading_symbol'],'expiry'])
        weekly = expiry[0]
        self.expiry = weekly

        self.lot_size = self.instruments.loc[(self.instruments['name']==self.config['trading_symbol']),'lot_size'].values[0]
        instrument_tokens = self.instruments.loc[(self.instruments['name']==self.config['trading_symbol'])&(self.instruments['expiry']==weekly)&((self.instruments['strike']>self.latest_ticks[self.index_tokens[self.config['trading_symbol']]]['last_price']-(5*self.config['strike_gap']))&(self.instruments['strike']<=self.latest_ticks[self.index_tokens[self.config['trading_symbol']]]['last_price']+(5*self.config['strike_gap'])))&(self.instruments['instrument_type']!='FUT'),'instrument_token'].tolist()
        self.instrument_tokens.extend(instrument_tokens)

        for token in self.instrument_tokens:
            self.trading_symbols[token] = self.instruments.loc[self.instruments['instrument_token']==token,'tradingsymbol'].values[0]
            self.strikes[token] = float(self.instruments.loc[self.instruments['instrument_token']==token,'strike'].values[0])

    def on_ticks(self, ws, ticks):
        
        if self.latest_ticks[self.index_tokens[self.config['trading_symbol']]] is not None:
            self.update_trading_symbol()
            ws.subscribe(self.instrument_tokens)

        for tick in ticks:
            self.tick_queue.put(tick)
        return
    
    def on_connect(self,ws):
        """Callback when WebSocket connects."""
        print("WebSocket connected!")
        ws.subscribe(self.instrument_tokens)
        ws.set_mode(ws.MODE_FULL, self.instrument_tokens)
        return
    

    def start(self):
        """Start the WebSocket connection."""
        self.kws.on_ticks = self.on_ticks
        self.kws.on_connect = self.on_connect

        # Start the tick processing thread
        tick_thread = threading.Thread(target=self.process_ticks)
        tick_thread.daemon = True
        tick_thread.start()

        # Start the WebSocket connection
        print("Connecting websocket")
        self.kws.connect(threaded=True)

        try:
            while True:
                time.sleep(5)  # Keep the main thread alive
                print(self.iv)
                print(self.delta)
                print(self.gamma)
        except KeyboardInterrupt:
            logging.info("Stopping the reactor...")
            self.running = False

        return
    


In [10]:
config = {    
    'exchange': 'NFO',
    'trading_symbol': 'NIFTY',
    'strike_gap': 50,
}

In [ ]:
sample = DeltaStrategy(config)
sample.start() 
# sample.instrument_tokens
# print(sample.api_key,sample.access_token)

Connecting websocket
Error processing tick: 
Error processing tick: 
Error processing tick: 
Error processing tick: 
{}
{}
{}
Error processing tick: 
Error processing tick: 


In [4]:
delt={'NIFTY2571025450PE': 'nan', 'NIFTY2571025600PE': 'nan', 'NIFTY2571025550CE': 0.42186085314098143, 'NIFTY2571025400CE': 0.607490946416877, 'NIFTY2571025300CE': 0.7242131284214952, 'NIFTY2571025350PE': 'nan', 'NIFTY2571025550PE': 'nan', 'NIFTY2571025500PE': 'nan', 'NIFTY2571025700PE': 'nan', 'NIFTY2571025500CE': 0.4829269583228646, 'NIFTY2571025650PE': 'nan', 'NIFTY2571025700CE': 0.25730058521889765, 'NIFTY2571025650CE': 0.3082863070590757, 'NIFTY2571025250PE': 'nan', 'NIFTY2571025400PE': 'nan', 'NIFTY2571025250CE': 0.7763489476151785, 'NIFTY2571025600CE': 0.36310749101315676, 'NIFTY2571025350CE': 0.6665368468901851, 'NIFTY2571025450CE': 0.5455091492682859, 'NIFTY2571025300PE': 'nan'}
delt


{'NIFTY2571025450PE': 'nan',
 'NIFTY2571025600PE': 'nan',
 'NIFTY2571025550CE': 0.42186085314098143,
 'NIFTY2571025400CE': 0.607490946416877,
 'NIFTY2571025300CE': 0.7242131284214952,
 'NIFTY2571025350PE': 'nan',
 'NIFTY2571025550PE': 'nan',
 'NIFTY2571025500PE': 'nan',
 'NIFTY2571025700PE': 'nan',
 'NIFTY2571025500CE': 0.4829269583228646,
 'NIFTY2571025650PE': 'nan',
 'NIFTY2571025700CE': 0.25730058521889765,
 'NIFTY2571025650CE': 0.3082863070590757,
 'NIFTY2571025250PE': 'nan',
 'NIFTY2571025400PE': 'nan',
 'NIFTY2571025250CE': 0.7763489476151785,
 'NIFTY2571025600CE': 0.36310749101315676,
 'NIFTY2571025350CE': 0.6665368468901851,
 'NIFTY2571025450CE': 0.5455091492682859,
 'NIFTY2571025300PE': 'nan'}